Sequential Quadratic Programming

Dependencies: `jax`, `scipy`, `numpy`

In [18]:
import jax
import jax.numpy as jnp
from jax import jacfwd, jacrev, hessian

from scipy.optimize import minimize, NonlinearConstraint

import numpy as np
from src import *
print(test())

NameError: name 'test' is not defined

Define your own Objective Functions and Constrains here :

(we use rosenbrock function here as an example)

$f_N(x)=\sum^{N-1}_{i=1}100(x_{i+1}-{x_i}^2)^2+(1-x_i)^2$

\begin{equation*}
\begin{split}
&\min_{x} \,\, f_2(x)\\
&s.t.\quad  \left\{\begin{array}{lc}
1-x_0-2x_1\geq 0\\
1-x_0^2-x_1\geq 0\\
1-x_0^2+x_1\geq 0\\
\end{array}\right.
\end{split}
\end{equation*}

In [2]:
#the rosenbrock function
def f(x):
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)
#constrains
def cons(x):
    return [1-x[0]-2*x[1],1-x[0]**2-x[1],1-x[0]**2+x[1]]

In [ ]:
def rosen_der(x):
    return np.array()